# jsonpickle

the elegent json serialisation library

In [1]:
pip install jsonpickle

Note: you may need to restart the kernel to use updated packages.


## directory()

simple object browser for diagnostics purpose

In [2]:
def directory(object):
    print(type(object))
    for key in dir(object):
        if key.startswith('_'):
            continue
        value = getattr(object, key)
        print('\t@%s: %s = %s'%(key, type(value), value))

## perdy

another simple function to print the object perdy like

In [3]:
def perdy(obj):
    keys = filter(lambda x: not x.startswith('_'), dir(obj))
    d = dict(map(lambda x: (x, getattr(obj,x)), keys))
    print('name={name}\ndob={dob}\nage={age}'.format(**d))

# import

In [4]:
import json
from datetime import datetime, time
from jsonpickle import encode, decode

## Person

a base class to pickle

In [5]:
class Person(object):
    '''
    base class for all people types
    '''
    
    def __init__(self,
        name=None,
        dob=None,
    ):
        self.name = name
        self.dob = dob
    
    @property
    def age(self):
        return (datetime.now() - self.dob).days/365.25

## Parent

that extends Person and links to Child

In [6]:
class Parent(Person):
    '''
    lets test inheritance
    '''

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.children = set()

    def add_child(self, child):
        self.children.add(child)
        child.parents.add(self)

## Child

that extends Person and links to Parent

In [7]:
class Child(Person):
    '''
    lets test relationships
    '''

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.parents = set()

    def add_parent(self, parent):
        self.parents.add(parent)
        parent.children.add(self)

## parent

lets create a parent

In [8]:
dob = datetime.strptime("1970-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")
parent = Parent(name='Dad', dob=dob)
#directory(parent)
perdy(parent)

name=Dad
dob=1970-01-01 00:00:00
age=50.223134839151264


## child

create a child and link it to the parent

In [9]:
dob = datetime.strptime("2000-01-11 00:00:00", "%Y-%m-%d %H:%M:%S")
child = Child(name='Son', dob=dob)
parent.add_child(child)
#directory(child)
perdy(child)

name=Son
dob=2000-01-11 00:00:00
age=20.197125256673512


### encode

create a json string representation of the object

In [10]:
pip install Perdy

Note: you may need to restart the kernel to use updated packages.


In [11]:
js = json.loads(encode(parent))

from Perdy.pretty import prettyPrintLn

prettyPrintLn(js)

{
  "children" : {
    "py/set" : [
      {
        "dob" : {
          "__reduce__" : [
            {
              "py/type" : "datetime.datetime"
            },
            [
              "B9ABCwAAAAAAAA=="
            ]
          ],
          "py/object" : "datetime.datetime"
        },
        "name" : "Son",
        "parents" : {
          "py/set" : [
            {
              "py/id" : 0
            }
          ]
        },
        "py/object" : "__main__.Child"
      }
    ]
  },
  "dob" : {
    "__reduce__" : [
      {
        "py/type" : "datetime.datetime"
      },
      [
        "B7IBAQAAAAAAAA=="
      ]
    ],
    "py/object" : "datetime.datetime"
  },
  "name" : "Dad",
  "py/object" : "__main__.Parent"
}


### decode

re-construct a python class from json string
make sure you put classes=[] to reconstitute the correct classes

In [12]:
spouse = decode(json.dumps(js), classes=[Parent, Child])
#directory(spouse)
perdy(spouse)

name=Dad
dob=1970-01-01 00:00:00
age=50.223134839151264
